In [1]:
#import packages
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape, Point
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request, json
import requests
import urllib.parse
import datetime

import warnings
warnings.filterwarnings('ignore')

In [3]:
access = pd.read_csv('../Data/tracts_subway_bus_access.csv', index_col = 0)
access.head()

,ctlabel,borocode,boroname,ct2020,boroct2020,ntaname,nta2020,cdta2020,cdtaname,geoid,shape_leng,shape_area,geometry,Bus_Access,Subway_Access,Bus_Count,Subway_Count
0,1.00,1,Manhattan,100,1000100,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,11023.048501,1.844421e+06,MULTIPOLYGON (((-74.04387761639944 40.69018767...,0,0,0,0
1,2.01,1,Manhattan,201,1000201,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4754.495247,9.723121e+05,MULTIPOLYGON (((-73.98449647076764 40.70950966...,1,1,23,3
2,6.00,1,Manhattan,600,1000600,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,6976.286456,2.582705e+06,MULTIPOLYGON (((-73.99022012669926 40.71440493...,1,1,25,4
3,14.01,1,Manhattan,1401,1001401,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,MULTIPOLYGON (((-73.9883662631772 40.716445702...,1,1,26,4
4,14.02,1,Manhattan,1402,1001402,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06,MULTIPOLYGON (((-73.98507342254645 40.71908329...,1,1,30,4


In [4]:
len(access)

2325

In [5]:
len(access[access['Bus_Access'] == 1])

2295

In [6]:
len(access[access['Subway_Access'] == 1])

1227

In [10]:
len((access[(access['Subway_Access'] == 1)|(access['Bus_Access'] == 1)]))

2298